In [1]:
# LOAD THE PACKAGES
import pandas as pd
import os
import time
import requests
import json
import csv
from tqdm import tqdm

import tweepy

import json
from logging import raiseExceptions 
import tweepy
import time
from datetime import datetime
import time
import os

import requests
import pandas as pd
import os

In [2]:
# LOAD API KEYS
consumer_key        = 'Z3BXr1e4ApQjojRMzDGXn3Kma'
consumer_secret     = '6N0o3oO8WwUiBF5jcjmDwjHsuCSZVyLaPfkw8CvdXoJpMzuzYj'
access_token        = '4075947982-96AArMZgSwsjMT471Su3ItkFXc3XnEOPvZcA879'
access_token_secret = 'Peb41wtWLS9tPJ9HqPlhJMJHAgpnMIjdsC8EbuyB3cUVZ'
bearer_token        = 'AAAAAAAAAAAAAAAAAAAAAHsnfAEAAAAAuulr34faSC6vEP7AE%2FnLgwpUiiA%3DSsRJsgAC4GfA5lkFGqC5KkOXoNPTpyayRlpLD6Fl2K97PU8D5n'

In [3]:
# DEFINE FUNCTION TO SAVE TWEEPY SEARCH RESULTS
#   searches=array with various tweepy search objects

def save_search_tweets_results(searches,info_str="",output_name="tweet-search.json"):
    if(str(type(searches)) == "<class 'list'>"):
        #COMBINE ALL JSONS FOR VARIOUS TWEETS INTO ON BIG JSON CALLED "out"
        out={}
        out["search_info"]=info_str
        #LOOP OVER SEARCHES
        tweet_ids=[]
        k=0 #counter
        for search in searches:
            #LOOP OVER TWEETS IN SEARCH
            for i in range(0,len(search)):
                out[str(k)]=search[i]._json
                tweet_id=search[i]._json["id_str"]
                #CHECK FOR REDUNDANT TWEETS
                if tweet_id in tweet_ids:
                    print("WARNING: REPEATED TWEETS IN SAVED FILE; ID = ",tweet_id)
                tweet_ids.append(search[i]._json["id_str"])
                k+=1
            #pretty_print_json(out)

        #DELETE FILE IF IT EXIST (START FRESH)
        if os.path.exists(output_name):
            os.remove(output_name)
        #WRITE FILE
        with open(output_name, 'w') as f:
            json.dump(out, f)
    else: 
        raise RuntimeError("ERROR: Incorrect datatype")

In [4]:
# SET UP CONNECTION
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) 

In [ ]:
# RUN SEARCH 

#SEARCH PARAM
query= "Worldcup" #keyword
fromDate = "2021-09-13" # Date of takeover
toDate = "2021-09-14" # Date of end of takeover


# NUMBER OF TWEETS TO SEARCH 
number_of_tweets=250
start_time = time.time()
max_loop_time_hrs=5
num_tweets_collected=0
searches=[]
k=0

#KEEP SEARCHING UNTIL DESIRED NUMBER OF TWEETS COLLECTED
while num_tweets_collected<number_of_tweets: 
    try: 
        #FIRST SEARCH
        if len(searches)==0:
            search_results = api.search_full_archive(query, fromDate, toDate, count=100)
        #ADDITIONAL SEARCHES
        else:
            search_results = api.search_full_archive(query, fromDate, toDate, count=100,max_id=max_id_next)

        #UPDATE PARAMETERS
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        #SAVE SEARCH RESULTS
        searches.append(search_results)

        #SAVE TEMPORARY CHECKPOINTS (DONT DO TOO OFTEN .. SLOWS CODE DOWN)
        if(k%10==0):
            print("SEARCH-"+str(k)+" COMPLETED;  TWEETS_COLLECTED=",num_tweets_collected,"; TIME (s) = ",time.time() - start_time)
        if(k%25==0):
            save_search_tweets_results(searches,output_name="tmp-snapshot.json")
            
        k+=1
    except:
        print("WARNING: twitter search failed")

    #SLEEP 5 SECONDS BEFORE NEXT REQUEST 
    if(number_of_tweets>18000):
        time.sleep(5)
    else:
        time.sleep(0.2)

# REPORT BASIC SEARCH INFO
print(num_tweets_collected,len(searches))
print("search time (s) =", (time.time() - start_time))

#TIMESTAMP SEARCH 
now = datetime.now()
dt_string = now.strftime("%Y-%m-%Y-H%H-M%M-S%S")

# SAVE RESULTS
info_str="query = "+query+"; number_of_tweets = "+str(number_of_tweets)+"; date = "+str(dt_string)
out_name=str(dt_string)+"-twitter-search.json"
save_search_tweets_results(searches,info_str=info_str,output_name=out_name)

#CLEAN-UP TEMP FILES
os.remove("tmp-snapshot.json")

In [ ]:
# NAEP Data Service API



In [ ]:
# Twitter API for those school district names + education in the years they were taken over by the state